In [1]:
import matplotlib.pyplot as plt
from collections import defaultdict

# Алгоритм NAMOA*

NAMOA* является основой для большинства алгоритмов многокритериального поиска. 

In [2]:
class Transition:
    def __init__(self, start_state, end_state, cost1, cost2):
        self.start_state = start_state
        self.end_state = end_state
        self.cost1 = cost1
        self.cost2 = cost2

In [3]:
def NAMOAstar(search_graph: SearchGraph,  start_state, goal_state, heuristic_func, search_tree: SearchTreePQD,):
    sols = ParetoSet()
    g_opened = defaultdict(ParetoSet)
    g_closed = defaultdict(ParetoSet)
    parent = defaultdict(set)

    g_opened[start_state].add_solution((0, 0))
    start_node = Node(start_state, (0, 0), (0, 0))

    search_tree = SearchTreePQD()
    search_tree.add_to_open(start_node)

    while not search_tree.open_is_empty():
        s = search_tree.get_best_node_from_open()
        g_opened[s.state].del_solution(s.g)
        g_closed[s.state].add_solution(s.g)

        if s.state == goal_state:
            sols.add_solution(s.g)
            # search_tree.remove_worse_nodes(s) ОШИБКА ИЗ СТАТЬИ
            continue

        for t in search_graph.get_neighbors(s.state):
            # t = [vertex_id, (cost1, cost2)]
            t_state =  t[0]
            t_g = tuple(x + y for x, y in zip(s.g, t[1]))
            t = Node(t_state, t_g, heuristic_func(t_state))
            
            if t.g in g_opened[t.state] or t.g in g_closed[t.state]:
                parent[t.g].add(s.g)
                continue

            if not g_opened[t_state].is_non_dominated(t.g) or not g_closed[t_state].is_non_dominated(t.g):
                continue

            if not sols.is_non_dominated(t.f):
                continue

            dominated_solutions = g_opened[t.state].all_solutions_dominated_by(t.g)
            g_opened[t.state].remove_worse(t.g)

            for sol in dominated_solutions:
                search_tree.remove_worse_opened(t.state, sol.solution_values)
            
            g_closed[t_state].remove_worse(t.g)

            parent[t.g].add(s.g)
            g_opened[t.state].add_solution(t.g)
            search_tree.add_to_open(t)
            
    return sols